### This notebook checks the QE curves as they are in obs_lsst_data repo

This is a modified version of the notebook by Simon Krughoff
https://github.com/lsst-sitcom/notebooks/blob/master/contrib/get_QE_curve.ipynb

Simon's notebook looks at all the QE curves that are in the ts8/CALIB. 
Here we specify a raft name (e.g. R22), then only plot the 9 CCDs for that raft.

Other added features:
* compare to the design curve in syseng_throughput, and 
* plot the raw data points so that we can examine the interpolation/extrapolation. 
 - We don't want to make the plots too busy so we only show raw data points averaged over the amps.

In [1]:
# make sure this is w_2019_49 or later
! eups list | grep lsst_distrib 

lsst_distrib          19.0.0+15  	current w_2019_52 setup


In [2]:
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, export_png
output_notebook()

from lsst.daf.persistence import Butler, NoResults
import os, numpy
import pandas as pd

Loading BokehJS ...

In [3]:
import lsst.syseng.throughputs as st
from lsst.geom import Point2D, Point2I
from lsst.afw.cameraGeom import FIELD_ANGLE, PIXELS
from lsst.obs.lsst.lsstCamMapper import LsstCamMapper

In [4]:
#rname = 'R10'
rname = 'R22'

In [5]:
DATADIR = f"{os.environ['OBS_LSST_DIR']}/ts8/CALIB"
#DATADIR = f"{os.environ['OBS_LSST_DIR']}/lsstcam/CALIB" #we can use this after DM-22605 gets merged
print(DATADIR)
butler = Butler(DATADIR)
cam = butler.get('camera')

/home/bxin/lsst_stack/obs_lsst/ts8/CALIB


In [6]:
# mapping based on 
#https://confluence.slac.stanford.edu/pages/viewpage.action?spaceKey=LSSTCAM&title=Raft+Delivery+and+Acceptance+Testing+Status
dd = pd.read_csv('raftInstall.csv',index_col=0)
rtmname = dd.rtm.loc[rname]
vendor = dd.vendor[rname]
print(rname, rtmname,vendor)

R22 RTM-024 e2v


### Retrieve the vendor design curve (e2v or ITL)

In [7]:
defaultDirs = st.setDefaultDirs()
vendorDir = defaultDirs['detector']+'/../'+vendor.lower()
addLosses = False 
detector = st.buildDetector(vendorDir, addLosses)

In [14]:
def make_subplot(plot, n):
    xmin = None
    xmax = None
    interp_list = []
    for k in qe_curve.data:
        wlen = qe_curve.data[k][0]
        #print(wlen)
        eff = qe_curve.data[k][1]    
        plot.circle(wlen.value, eff.value, line_color='black', line_width=3, legend_label='Amp data')
        
        amp_point = det[k].getBBox().getCenter()
        #print(det[k].getName())
        if xmin is None or xmax is None:
            xmin = wlen.min()
            xmax = wlen.max()
            interp_wlen = numpy.linspace(xmin, xmax, num=500)
        interp_eff = qe_curve.evaluate(det, amp_point, interp_wlen, kind='quadratic')
        #print(interp_wlen)
        interp_list.append(interp_eff)
        
    interp_arr = numpy.array([arr for arr in interp_list])
    median_eff = numpy.median(interp_arr, axis=0)
    #print(interp_arr.shape)
    plot.line(interp_wlen, median_eff, color='black', line_width=2, legend_label = 'Amp median')
    plot.line(detector.wavelen, detector.sb*100, color='red', line_width=2, legend_label = '%s design'%vendor)
    plot.legend.location = "center"
    plot.legend.click_policy="hide"
    
    if n > 6:
        plot.xaxis.axis_label = 'wavelength (nm)'
        
    if n in (1, 4, 7):
        plot.yaxis.axis_label = 'efficiency (%)'
        
    plot.text(x=[320,], y=[20,], text=[f'{det.getName().replace(rtmname, rname)}',])
    # plot.text(x=[320,], y=[20,], text=[f'{det.getName()}',]) #use this line after DM-22605 gets merged
    plot.text(x=[320,], y=[10,], text=[f'{rtmname}',])

In [15]:
plots = []
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rtmname:
    #if rname1 != rname: #use this line after DM-22605 gets merged
        continue;

    try:
        qe_curve = butler.get('qe_curve', raftName=rtmname, detectorName=dname, taiObs='2000-01-01T00:00:00')
        #qe_curve = butler.get('qe_curve', raftName=rname, detectorName=dname, taiObs='2000-01-01T00:00:00') #DM-22605 gets merged
    except NoResults:
        continue  # No results found for this detector

    if len(plots) > 0:
        plots.append(figure(x_range=plots[0].x_range, y_range=plots[0].y_range))
    else:
        plots.append(figure())
    print(rname1, rname)
    make_subplot(plots[-1], len(plots))

RTM-024 R22
RTM-024 R22
RTM-024 R22
RTM-024 R22
RTM-024 R22
RTM-024 R22
RTM-024 R22
RTM-024 R22
RTM-024 R22


In [16]:
grid = gridplot(plots, ncols=3, plot_width=350, plot_height=200)
show(grid)

In [17]:
pltDir = os.path.join('dm_plots', rname)
if not os.path.exists(pltDir):
    os.mkdir(pltDir)
pngPath = os.path.join(pltDir, 'QE_%s.png'%rname)
export_png(grid, filename=pngPath)

'/home/bxin/notebooks/scratchBook/m5_by_amp/dm_plots/R22/QE_R22.png'